In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Row, DataFrame

from hops import hdfs
from tempfile import TemporaryFile

import pandas as pd
import numpy as np

import math

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
44,application_1541669638743_0147,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
part3_path = "hdfs:///Projects/traffic_reginbald/processed_traffic_data/partitions/backward_partitions-3min.csv"
part5_path = "hdfs:///Projects/traffic_reginbald/processed_traffic_data/partitions/backward_partitions-5min.csv"
part10_path = "hdfs:///Projects/traffic_reginbald/processed_traffic_data/partitions/backward_partitions-10min.csv"
part20_path = "hdfs:///Projects/traffic_reginbald/processed_traffic_data/partitions/backward_partitions-20min.csv"
part30_path = "hdfs:///Projects/traffic_reginbald/processed_traffic_data/partitions/backward_partitions-30min.csv"


overlap_path = "hdfs:///Projects/traffic_reginbald/processed_traffic_data/partitions/overlapping_partitions-base_weight_2_min-forward_3-backward_10.csv"

In [36]:
partition_schema = StructType().add('node', StringType(), False) \
        .add('partition', ShortType(), False) \
        .add('group', StringType(), False) 
        
overlap_schema = StructType().add('node', StringType(), False) \
        .add('partition', ShortType(), False) \
        .add('type', StringType(), False) 

In [37]:
part3_df = spark.read.csv(part3_path, sep=';', schema=partition_schema, ignoreLeadingWhiteSpace=True, \
                    ignoreTrailingWhiteSpace=True, timestampFormat='yyyy-MM-dd HH:mm:ss.SSS').where(~isnull("partition"))
part5_df  = spark.read.csv(part5_path, sep=';', schema=partition_schema, ignoreLeadingWhiteSpace=True, \
                    ignoreTrailingWhiteSpace=True, timestampFormat='yyyy-MM-dd HH:mm:ss.SSS').where(~isnull("partition"))
part10_df = spark.read.csv(part10_path, sep=';', schema=partition_schema, ignoreLeadingWhiteSpace=True, \
                    ignoreTrailingWhiteSpace=True, timestampFormat='yyyy-MM-dd HH:mm:ss.SSS').where(~isnull("partition"))
part20_df = spark.read.csv(part20_path, sep=';', schema=partition_schema, ignoreLeadingWhiteSpace=True, \
                    ignoreTrailingWhiteSpace=True, timestampFormat='yyyy-MM-dd HH:mm:ss.SSS').where(~isnull("partition"))
part30_df = spark.read.csv(part30_path, sep=';', schema=partition_schema, ignoreLeadingWhiteSpace=True, \
                    ignoreTrailingWhiteSpace=True, timestampFormat='yyyy-MM-dd HH:mm:ss.SSS').where(~isnull("partition"))
overlap_df = spark.read.csv(overlap_path, sep=';', schema=overlap_schema, ignoreLeadingWhiteSpace=True, \
                    ignoreTrailingWhiteSpace=True, timestampFormat='yyyy-MM-dd HH:mm:ss.SSS').where(~isnull("partition"))

In [38]:
print part3_df.agg({"partition": "max"}).collect()[0]
print part5_df.agg({"partition": "max"}).collect()[0]
print part10_df.agg({"partition": "max"}).collect()[0]
print part20_df.agg({"partition": "max"}).collect()[0]
print part30_df.agg({"partition": "max"}).collect()[0]

Row(max(partition)=35)
Row(max(partition)=18)
Row(max(partition)=12)
Row(max(partition)=6)
Row(max(partition)=6)

In [39]:
print part3_df.groupBy("partition").count().agg({"count": "avg"}).collect()[0]
print part5_df.groupBy("partition").count().agg({"count": "avg"}).collect()[0]
print part10_df.groupBy("partition").count().agg({"count": "avg"}).collect()[0]
print part20_df.groupBy("partition").count().agg({"count": "avg"}).collect()[0]
print part30_df.groupBy("partition").count().agg({"count": "avg"}).collect()[0]

Row(avg(count)=23.805555555555557)
Row(avg(count)=45.10526315789474)
Row(avg(count)=65.92307692307692)
Row(avg(count)=122.42857142857143)
Row(avg(count)=122.42857142857143)

In [40]:
print part3_df.groupBy("partition").count().agg({"count": "max"}).collect()[0]
print part5_df.groupBy("partition").count().agg({"count": "max"}).collect()[0]
print part10_df.groupBy("partition").count().agg({"count": "max"}).collect()[0]
print part20_df.groupBy("partition").count().agg({"count": "max"}).collect()[0]
print part30_df.groupBy("partition").count().agg({"count": "max"}).collect()[0]

Row(max(count)=194)
Row(max(count)=245)
Row(max(count)=408)
Row(max(count)=678)
Row(max(count)=678)

In [41]:
print part3_df.groupBy("partition").count().agg({"count": "min"}).collect()[0]
print part5_df.groupBy("partition").count().agg({"count": "min"}).collect()[0]
print part10_df.groupBy("partition").count().agg({"count": "min"}).collect()[0]
print part20_df.groupBy("partition").count().agg({"count": "min"}).collect()[0]
print part30_df.groupBy("partition").count().agg({"count": "min"}).collect()[0]

Row(min(count)=1)
Row(min(count)=1)
Row(min(count)=1)
Row(min(count)=11)
Row(min(count)=11)

In [49]:
print overlap_df.agg({"partition": "max"}).collect()[0]
print overlap_df.groupBy("partition").count().agg({"count": "avg"}).collect()[0]
print overlap_df.groupBy("partition").count().agg({"count": "max"}).collect()[0]
print overlap_df.groupBy("partition").count().agg({"count": "min"}).collect()[0]

Row(max(partition)=61)
Row(avg(count)=268.9032258064516)
Row(max(count)=609)
Row(min(count)=11)

In [51]:
print part3_df.groupBy("node").count().collect()[0]

Row(node=u'E6N-12920', count=1)